# side quest

## Sampling dimensions

An optical systems is defined with a model that represents a physical reality where there are infinitely many rays of light emanating from light sources.

In practice, when simulating, we need to decide how many rays we use for computation. This is called **sampling** and the number of rays is also called the number of samples.

This is not to say that rays exist for all possible values for the ray variables. Sometimes they follow a particular distribution, like wavelength can be between 400 and 600nm only ; angular distribution can be restricted to [-10, 10]°, and so on. But those distributions are continuous, and infinitly many rays can follow them.

A system defines multiple dimensions, corresponding to different continuous variables that uniquely describe a ray. For example, the coordinate of the source point of a ray is one possible dimension. Another one is the wavelength of the ray.

* Base dimension: This dimension is always present unless we simulate a single ray. It's the dimension along which we sample when all other variables are fixed.

-> rename to angular? at infinity angular becomes linear, but really it's still the emanating angle of the ray from the object

can actually not exist if simulating a single ray
  
* Object dimension
* Wavelength dimension
* System configuration

Additionally, the number of spatial dimension can also be 2 or 3. When doing a 2D simulation, rays are restricted to a single meridional plane and don't have a z coordinate. This simplifies the simulation considerably, but at the price of ignoring skew rays.

Note that the meridional plane in the 2D simulation is abstract and not necessarily the Z=0 plane. If the system is not rotationally symmetric, 2D mode is not possible.

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm


from typing import Any

Tensor = torch.Tensor


def view(optics, sampling):
    scene = tlm.viewer.render_sequence(optics, sampling)
    #tlm.viewer.dump(scene, ndigits=2)
    tlm.viewer.show(scene)

surface = tlm.surfaces.Parabola(15.0, 0.020)
surface2 = tlm.surfaces.Parabola(15.0, 0.030)
surface3 = tlm.surfaces.Sphere(12, -20)

optics = nn.Sequential(
    tlm.PointSourceAtInfinity(18., 0),
    tlm.Gap(5.0),
    tlm.OpticalSurface(surface, anchors=("origin", "extent")),
    tlm.Gap(1.0),
    tlm.OpticalSurface(surface2, scale=-1, anchors=("extent", "origin")),
    #tlm.Gap(0.5),
    #tlm.OpticalSurface(surface3, anchors=("extent", "extent")),
    #tlm.Gap(5.0),
    #tlm.OpticalSurface(surface3, scale=-1, anchors=("extent", "extent")),
)


sampling = {"dim": 2, "dtype": torch.float64, "base": 10}
view(optics, sampling)

sampling = {"dim": 3, "dtype": torch.float64, "base": 10}
view(optics, sampling)

In [ ]:
# mode 1:  inline / chained / affecting
# surface transform = input.transforms - anchor + scale
# output transform = input.transforms - first anchor + second anchor

# mode 2: offline / free / independent
# surface transform = input.transforms + local transform - anchor + scale
# output transform = input.transforms

# how to support absolute position on chain?

# RS(X - A) + T
# surface transform(X) = CSX - A
# surface transform = anchor1 + scale + chain
# output transform = chain + anchor1 + anchor2